In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('../bigdata/vgsales.csv')

In [4]:
def discretization(attr):
    inp = list(df[attr].to_numpy())
    midi = df[attr].describe()
    out = []
    for x in inp:
        if(x<=float(midi.loc['25%'])):
            out.append(attr+" sangat kecil")
        elif(x>float(midi.loc['25%']) and x<=float(midi.loc['50%'])):
            out.append(attr+" kecil")
        elif(x>float(midi.loc['50%']) and x<=float(midi.loc['75%'])):
            out.append(attr+" sedang")
        else:
            out.append(attr+" besar")
    return out
def multiDiscretization(attrs):
    for x in attrs:
        df[x+"_desc"] = pd.Series(discretization(x),index=df.index)
multiDiscretization(['NA_Sales','EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'])

In [5]:
df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,NA_Sales_desc,EU_Sales_desc,JP_Sales_desc,Other_Sales_desc,Global_Sales_desc
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,NA_Sales besar,EU_Sales besar,JP_Sales besar,Other_Sales besar,Global_Sales besar
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NA_Sales besar,EU_Sales besar,JP_Sales besar,Other_Sales besar,Global_Sales besar
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,NA_Sales besar,EU_Sales besar,JP_Sales besar,Other_Sales besar,Global_Sales besar
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,NA_Sales besar,EU_Sales besar,JP_Sales besar,Other_Sales besar,Global_Sales besar
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NA_Sales besar,EU_Sales besar,JP_Sales besar,Other_Sales besar,Global_Sales besar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01,NA_Sales kecil,EU_Sales sangat kecil,JP_Sales sangat kecil,Other_Sales sangat kecil,Global_Sales sangat kecil
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01,NA_Sales kecil,EU_Sales sangat kecil,JP_Sales sangat kecil,Other_Sales sangat kecil,Global_Sales sangat kecil
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01,NA_Sales sangat kecil,EU_Sales sangat kecil,JP_Sales sangat kecil,Other_Sales sangat kecil,Global_Sales sangat kecil
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01,NA_Sales sangat kecil,EU_Sales kecil,JP_Sales sangat kecil,Other_Sales sangat kecil,Global_Sales sangat kecil


In [ ]:
df.columns

In [6]:
data = df[['Platform','Genre','Publisher']].astype(str).values.tolist()

In [35]:
data = df[['Platform','Genre','Publisher','NA_Sales_desc','EU_Sales_desc', 'JP_Sales_desc', 'Other_Sales_desc', 'Global_Sales_desc']].astype(str).values.tolist()

In [7]:
from sqlalchemy import create_engine
from pandas.io import sql
engine = create_engine('sqlite:///:memory:')
df.to_sql('vgsales',engine)

16598

In [9]:
df['Platform'].unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       '3DS', 'PS4', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne', 'GC',
       'WiiU', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [36]:
data

[['Wii',
  'Sports',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['NES',
  'Platform',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['Wii',
  'Racing',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['Wii',
  'Sports',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['GB',
  'Role-Playing',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['GB',
  'Puzzle',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['DS',
  'Platform',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales b

In [10]:
#tidak perlu dikerjakan yg ini
def ARMformat(a,b,tb):
    collect = []
    for x in df[a].unique():
        q = "SELECT "+b+" FROM "+tb+" WHERE "+a+"='"+x+"'"
        res = pd.read_sql_query(q,engine)
        res=res[b].unique()
        resclean = []
        for i in res:
            if i!=None:
                resclean.append(i)
        #resmin = resclean[0:10] #terpaksa biar cepat saja
        resmin = resclean
        collect.append(list(resmin))
    return collect

In [11]:
#tidak perlu dikerjakan yg ini
data = ARMformat('Platform','Publisher','vgsales')
len(data)

31

In [40]:
data

[['Wii',
  'Sports',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['NES',
  'Platform',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['Wii',
  'Racing',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['Wii',
  'Sports',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['GB',
  'Role-Playing',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['GB',
  'Puzzle',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales besar'],
 ['DS',
  'Platform',
  'Nintendo',
  'NA_Sales besar',
  'EU_Sales besar',
  'JP_Sales besar',
  'Other_Sales besar',
  'Global_Sales b

In [13]:
#tidak perlu dikerjakan yg ini
i = 0
pre = df['Platform'].unique()
for dt in data:
    print(pre[i],dt,"\n")
    i+=1

Wii ['Nintendo', 'Ubisoft', 'Sega', '505 Games', 'LucasArts', 'Activision', 'Take-Two Interactive', 'Electronic Arts', 'Warner Bros. Interactive Entertainment', 'Disney Interactive Studios', 'Majesco Entertainment', 'Konami Digital Entertainment', 'Midway Games', 'THQ', 'Capcom', 'MTV Games', 'Mindscape', 'Activision Value', 'Namco Bandai Games', 'Atari', 'Zoo Games', 'Deep Silver', 'RTL', 'D3Publisher', 'Tomy Corporation', 'Square Enix', 'Bethesda Softworks', 'Unknown', 'Codemasters', 'Eidos Interactive', 'Rising Star Games', 'Hudson Soft', 'Koch Media', 'Nordic Games', 'Big Ben Interactive', 'Funbox Media', 'Rondomedia', 'Vivendi Games', 'System 3 Arcade Software', 'Scholastic Inc.', 'XS Games', 'Xplosiv', 'GameMill Entertainment', 'Knowledge Adventure', 'Mastiff', 'Pinnacle', 'Destineer', 'Game Factory', 'Zushi Games', 'Zoo Digital Publishing', 'City Interactive', 'Storm City Games', 'Little Orbit', 'DTP Entertainment', 'Oxygen Interactive', 'Focus Home Interactive', 'Detn8 Games', 

FPGrowth

In [ ]:
!pip install pyfpgrowth

In [14]:
import pyfpgrowth

In [38]:
patterns = pyfpgrowth.find_frequent_patterns(data,200)
print(len(patterns.items()))
for p,d in patterns.items():
    if len(p)>4:
        print(p,":",d)

1645
('EU_Sales besar', 'JP_Sales besar', 'NA_Sales besar', 'Nintendo', 'Other_Sales besar') : 235
('EU_Sales besar', 'Global_Sales besar', 'NA_Sales besar', 'Nintendo', 'Other_Sales besar') : 239
('EU_Sales besar', 'Global_Sales besar', 'JP_Sales besar', 'NA_Sales besar', 'Nintendo', 'Other_Sales besar') : 235
('EU_Sales besar', 'Global_Sales besar', 'JP_Sales besar', 'Nintendo', 'Other_Sales besar') : 240
('Global_Sales besar', 'JP_Sales besar', 'NA_Sales besar', 'Nintendo', 'Other_Sales besar') : 252
('EU_Sales besar', 'Global_Sales besar', 'JP_Sales besar', 'NA_Sales besar', 'Nintendo') : 307
('EU_Sales kecil', 'GBA', 'JP_Sales sangat kecil', 'NA_Sales kecil', 'Other_Sales sangat kecil') : 225
('GBA', 'Global_Sales sangat kecil', 'JP_Sales sangat kecil', 'NA_Sales kecil', 'Other_Sales sangat kecil') : 218
('Global_Sales sangat kecil', 'JP_Sales sangat kecil', 'NA_Sales kecil', 'Other_Sales sangat kecil', 'XB') : 201
('EU_Sales kecil', 'JP_Sales sangat kecil', 'NA_Sales kecil', 'Oth

In [43]:
rules = pyfpgrowth.generate_association_rules(patterns,0.2)
for p,d in rules.items():
    """if len(p)>1:
        if d[0][0]=='PS2':
            print(p,"=>",d)
    if "JP_Sales" in d[0][0]:
        print(p,"=>",d)"""
    print(p,"=>",d)

('Disney Interactive Studios',) => (('JP_Sales sangat kecil',), 0.9357798165137615)
('N64',) => (('JP_Sales sangat kecil',), 0.6959247648902821)
('Atari',) => (('JP_Sales sangat kecil',), 0.8732782369146006)
('Take-Two Interactive',) => (('JP_Sales sangat kecil',), 0.8329297820823245)
('EU_Sales sangat kecil', 'PSV') => (('NA_Sales sangat kecil',), 0.9070631970260223)
('NA_Sales sangat kecil', 'PSV') => (('EU_Sales sangat kecil',), 0.8905109489051095)
('Global_Sales sangat kecil', 'NA_Sales sangat kecil', 'PSV') => (('Other_Sales sangat kecil',), 0.9853658536585366)
('Global_Sales sangat kecil', 'Other_Sales sangat kecil', 'PSV') => (('NA_Sales sangat kecil',), 0.9853658536585366)
('NA_Sales sangat kecil', 'Other_Sales sangat kecil', 'PSV') => (('EU_Sales sangat kecil',), 0.9494163424124513)
('EU_Sales sangat kecil', 'NA_Sales sangat kecil', 'PSV') => (('Other_Sales sangat kecil',), 1.0)
('EU_Sales sangat kecil', 'Other_Sales sangat kecil', 'PSV') => (('NA_Sales sangat kecil',), 0.9838

Apriori

In [ ]:
!pip install efficient_apriori

In [44]:
from efficient_apriori import apriori

In [45]:
itemsets, rules = apriori(data, min_support=0.01,  min_confidence=1.0)

In [46]:
hasil = []
for x in rules:
    hasil.append([str(x.lhs)+"=>"+str(x.rhs),x.support,x.confidence,x.lift,x.conviction])
    print(str(x.lhs)+"=>"+str(x.rhs),x.support,x.confidence,x.lift,x.conviction,sep="\t")

('EU_Sales besar', 'PC')=>('JP_Sales sangat kecil',)	0.012832871430292806	1.0	1.5875657580105211	370104831.90745866
('EU_Sales kecil', 'GC')=>('Other_Sales sangat kecil',)	0.011447162308711893	1.0	2.5626061448201325	609772261.7182792
('EU_Sales kecil', 'PS')=>('NA_Sales kecil',)	0.010181949632485842	1.0	4.038442822384429	752379804.7957585
('Global_Sales sangat kecil', 'JP_Sales besar')=>('EU_Sales sangat kecil',)	0.02193035305458489	1.0	2.8966841186736474	654777684.0583202
('NA_Sales sangat kecil', 'PS')=>('EU_Sales sangat kecil',)	0.012591878539583082	1.0	2.8966841186736474	654777684.0583202
('NA_Sales sangat kecil', 'SAT')=>('EU_Sales sangat kecil',)	0.010242197855163273	1.0	2.896684118673648	654777684.0583202
('EU_Sales sangat kecil', 'SAT')=>('NA_Sales sangat kecil',)	0.010242197855163273	1.0	3.6892642809513227	728943246.1742377
('NA_Sales sangat kecil', 'SNES')=>('EU_Sales sangat kecil',)	0.011808651644776479	1.0	2.8966841186736474	654777684.0583202
('Other_Sales sangat kecil', 'S

Tugas lakukan hal yang sama dengan menggunakan avocado sales
- lakukan discretization
- lakukan format data
- fp growth : panjang rule lebih dari 4, support: 0.05, confidence 1.0
- apriori : panjang rule lebih dari 4, support: 0.05, confidence 1.0